# QXQ YLC 2024 Capstone Project
<ul>
<li>by Priyanuj Kashyap</li>
<li>Student ID: 3824_Kashyap</li>
</ul>

## Introduction:
The two functions were built on the concept of BB84 Protocol for Quantum Key Distribution. The considerations take while the building the functions were.
<ol>
<li>The focus was put on the concept of BB84 Protocol and has been kept simple to understand.</li>
<li>Focus was also put into how an encrypted message can be shared and then decrypted on the other end.</li>
<li>The binary of the messages also have been shown to display how the message is encrypted and decrypted.</li>
</ol>


## Code Snippets

In [1]:
import cirq
from random import choices

def QKD(num_bits):

  #Setup
  encode_gates = {0: cirq.I, 1: cirq.X}
  basis_gates = {'Z': cirq.I, 'X': cirq.H}

  qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

  #Alice Chooses Bits and Bases
  alice_key = choices([0, 1], k = num_bits)
  alice_bases = choices(['Z', 'X'], k = num_bits)

  #Alice Creates Qubits
  alice_circuit = cirq.Circuit()

  for bit in range(num_bits):

    encode_value = alice_key[bit]
    encode_gate = encode_gates[encode_value]

    basis_value = alice_bases[bit]
    basis_gate = basis_gates[basis_value]

    qubit = qubits[bit]
    alice_circuit.append(encode_gate(qubit))
    alice_circuit.append(basis_gate(qubit))

  #Bob chooses a Bases
  bob_bases = choices(['Z', 'X'], k = num_bits)

  bob_circuit = cirq.Circuit()

  for bit in range(num_bits):

    basis_value = bob_bases[bit]
    basis_gate = basis_gates[basis_value]

    qubit = qubits[bit]
    bob_circuit.append(basis_gate(qubit))

  #Bob Measures Qubits
  bob_circuit.append(cirq.measure(qubits, key = 'bob key'))

  #Bob Creates a Key
  bb84_circuit = alice_circuit + bob_circuit

  sim = cirq.Simulator()
  results = sim.run(bb84_circuit)
  bob_key = results.measurements['bob key'][0]

  final_alice_key = []
  final_bob_key = []

  #Compare Bases
  for bit in range(num_bits):

    if alice_bases[bit] == bob_bases[bit]:
      final_alice_key.append(alice_key[bit])
      final_bob_key.append(bob_key[bit])

  #Compare Half their Bits
  num_bits_to_compare = int(len(final_alice_key) * .5)
  if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
    final_alice_key = final_alice_key[num_bits_to_compare:]
    final_bob_key = final_bob_key[num_bits_to_compare:]

    print('Alice Key: ', final_alice_key)
    print('Bob Key: ', final_bob_key)

  else:
    print('\n\nEve was listening, we need to use a different channel!')

In [2]:
#Message
unencrypted_string='Priyanuj'

In [17]:
#Keys
QKD(216)

Alice Key:  [0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1]
Bob Key:  [0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1]


In [4]:
#Encrypt Message
def encrypt(final_alice_key):
  binary_message = ''.join(format(ord(i), '08b') for i in unencrypted_string)
  key = ''.join(map(str,final_alice_key))
  encrypted_message = int((binary_message), 2) ^ int(key[:len(binary_message)], 2)
  encrypted_message = format(encrypted_message, '0' + str(len(binary_message)) + 'b')
  encrypted_string = ''.join(chr(int(encrypted_message[i:i+8], 2)) for i in range(0, len(encrypted_message), 8))
  print('Encrypted Message in Binary:',encrypted_message)
  print('Encrypted Message:',encrypted_string)

In [27]:
encrypt([0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1])

Encrypted Message in Binary: 0101101001000110001101111001011110001011001110011001101001110111
Encrypted Message: ZF79w


In [28]:
#Decrypt Message
def decrypt(encrypted_string,final_bob_key):
  binary_encrypted_message = ''.join(format(ord(i), '08b') for i in encrypted_string)
  key = ''.join(map(str,final_bob_key))
  decrypted_message = int(binary_encrypted_message, 2) ^ int(key[:len(binary_encrypted_message)], 2)
  decrypted_message = format(decrypted_message, '0' + str(len(binary_encrypted_message)) + 'b')
  print('Decrypted Message in Binary:',decrypted_message)
  decrypted_message = ''.join(chr(int(decrypted_message[i:i+8], 2)) for i in range(0, len(decrypted_message), 8))
  print('Decrypted Message:',decrypted_message)

In [29]:
decrypt('ZF79w',[0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1])

Decrypted Message in Binary: 0101000001110010011010010111100101100001011011100111010101101010
Decrypted Message: Priyanuj


## Analysis

In the BB84 Quantum Key Distribution (QKD) protocol, an eavesdropper, often referred to as Eve, can significantly affect the communication between Alice (the sender) and Bob (the receiver):
<ol>
<li> Intercept and Resend: One common strategy for eavesdropping is the intercept-and-resend attack. In this strategy, Eve intercepts the quantum bit (qubit) sent by Alice, measures it, and then sends a new qubit to Bob. However, due to the principles of quantum mechanics, this interception and measurement inevitably disturb the qubit and increase the quantum bit error rate (QBER). </li>

<li> Detection: The disturbance caused by Eve's interception can be detected by Alice and Bob when they compare a subset of their keys over a public channel. If the error rate exceeds a certain threshold, they know an eavesdropper is present. This is because it's impossible to distinguish between the errors caused by an eavesdropper and the errors produced by imperfections of the optical channel and optoelectronic devices.</li>

<li> Security: The security of the BB84 protocol is based on the fact that any attempt to gain information about the key by Eve will introduce detectable anomalies. Therefore, if Alice and Bob detect a higher error rate, they can abort the key and start a new communication.</li>

<li> Noise: It's important to note that in practical realizations, a QKD protocol is only secure when the QBER introduced by an eavesdropper unavoidably exceeds the system error rate. This condition guarantees that an eavesdropper cannot disguise her presence by simply replacing the original transmission line with a less faulty one.</li>
</ol>

So, while an eavesdropper can attempt to gain information during the quantum communication, the principles of quantum mechanics used in the BB84 protocol allow Alice and Bob to detect and mitigate such attempts

## Conclusion
The development of quantum encryption is crucial for the future of secure communication for several reasons:
<ol>
<li>Quantum Computing Threat: Quantum computers have the potential to break widely used encryption methods, rendering much of today’s secure communication vulnerable. This is because quantum computers could, in theory, factor incredibly large numbers efficiently, which could compromise the security of widely used encryption methods like RSA.</li>
<li>Quantum-Safe Cryptography: To counter this threat, researchers are developing quantum-safe cryptography, also known as post-quantum or quantum-proof cryptography. These new encryption methods aim to be secure against both classical and quantum computers.</li>
<li>Secure Key Distribution: Quantum Key Distribution (QKD) offers a potential solution to the challenges faced by traditional encryption methods. QKD uses the laws of physics to generate and distribute secure keys, making undetected eavesdropping impossible.</li>
</ol>